<a href="https://colab.research.google.com/github/danielburch/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [0]:
#House keeping
import pandas as pd
import numpy as np
from scipy import stats

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-06-09 20:20:24--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-06-09 20:20:24 (154 KB/s) - ‘student.zip’ saved [20478/20478]



In [0]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [0]:
df = pd.read_csv('student-mat.csv', sep=';')

print(df.shape)
df.head()


(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [0]:
#Dependent Variables - Hypothesis: Mother's education related to Mother's job

'''
Null Hypothesis: These two variables are "independent" there's no relationship between the two

Alternative Hypothesis: The two variables are "dependent" there is a relationship or association between them.
'''

In [0]:

df['Medu'].value_counts().sort_index()

0      3
1     59
2    103
3     99
4    131
Name: Medu, dtype: int64

In [0]:
df['Mjob'].value_counts().sort_index()

at_home      59
health       34
other       141
services    103
teacher      58
Name: Mjob, dtype: int64

In [0]:
con = pd.crosstab(df['Medu'], df['Mjob'])

con

Mjob,at_home,health,other,services,teacher
Medu,,,,,
0,2,0,1,0,0
1,25,1,28,5,0
2,22,2,47,32,0
3,8,5,43,40,3
4,2,26,22,26,55


In [0]:
chi2, p_value, dof, expected = stats.chi2_contingency(con)
print("Chi2 Stat",chi2)
print("P-value", p_value)
print("DOF",dof)
print("Eft", expected)

Chi2 Stat 224.6843492089439
P-value 7.752732260542515e-39
DOF 16
Eft [[ 0.44810127  0.25822785  1.07088608  0.78227848  0.44050633]
 [ 8.81265823  5.07848101 21.06075949 15.38481013  8.66329114]
 [15.38481013  8.86582278 36.76708861 26.85822785 15.12405063]
 [14.78734177  8.52151899 35.33924051 25.81518987 14.53670886]
 [19.56708861 11.27594937 46.76202532 34.15949367 19.23544304]]


In [0]:
'''
Conclusion: Based on a chi2 statistic of 224.68 and a p-value of 0, we reject the null hypothesis that Mother's 
education and Mother's job are independent. We accept the alternative hypothesis that these variables may have a 
dependent relationship between them

'''

In [0]:
#Independent Variables - Hypothesis: Student's sex not related to absences

'''
Null Hypothesis: These two variables are "independent" there's no relationship between the two

Alternative Hypothesis: The two variables are "dependent" there is a relationship or association between them.

In [0]:
df['sex'].value_counts().sort_index()

F    208
M    187
Name: sex, dtype: int64

In [0]:
df['absences'].value_counts().sort_index()

0     115
1       3
2      65
3       8
4      53
5       5
6      31
7       7
8      22
9       3
10     17
11      3
12     12
13      3
14     12
15      3
16      7
17      1
18      5
19      1
20      4
21      1
22      3
23      1
24      1
25      1
26      1
28      1
30      1
38      1
40      1
54      1
56      1
75      1
Name: absences, dtype: int64

In [0]:
con2 = pd.crosstab(df['sex'], df['absences'])

con2

absences,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,30,38,40,54,56,75
sex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F,63,2,32,2,27,2,19,5,8,1,10,0,8,0,7,2,3,1,2,0,2,1,2,1,1,1,1,1,0,0,1,1,1,1
M,52,1,33,6,26,3,12,2,14,2,7,3,4,3,5,1,4,0,3,1,2,0,1,0,0,0,0,0,1,1,0,0,0,0


In [0]:
chi2, p_value, dof, expected = stats.chi2_contingency(con2)
print("Chi2 Stat",chi2)
print("P-value", p_value)
print("DOF",dof)
print("Eft", expected)

Chi2 Stat 30.631541885461054
P-value 0.5855619885852188
DOF 33
Eft [[60.55696203  1.57974684 34.2278481   4.21265823 27.90886076  2.63291139
  16.32405063  3.68607595 11.58481013  1.57974684  8.95189873  1.57974684
   6.31898734  1.57974684  6.31898734  1.57974684  3.68607595  0.52658228
   2.63291139  0.52658228  2.10632911  0.52658228  1.57974684  0.52658228
   0.52658228  0.52658228  0.52658228  0.52658228  0.52658228  0.52658228
   0.52658228  0.52658228  0.52658228  0.52658228]
 [54.44303797  1.42025316 30.7721519   3.78734177 25.09113924  2.36708861
  14.67594937  3.31392405 10.41518987  1.42025316  8.04810127  1.42025316
   5.68101266  1.42025316  5.68101266  1.42025316  3.31392405  0.47341772
   2.36708861  0.47341772  1.89367089  0.47341772  1.42025316  0.47341772
   0.47341772  0.47341772  0.47341772  0.47341772  0.47341772  0.47341772
   0.47341772  0.47341772  0.47341772  0.47341772]]


In [0]:
'''
Conclusion: Based on a chi2 statistic of 30.63 and a p-value of .5855, we fail to reject the null hypothesis 
that student's sex and absences are independent. The variables may be independent

'''

### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [0]:
observed_with_margins = pd.crosstab(df['Medu'], df['Mjob'],margins=True)

observed_with_margins

Mjob,at_home,health,other,services,teacher,All
Medu,,,,,,
0,2,0,1,0,0,3
1,25,1,28,5,0,59
2,22,2,47,32,0,103
3,8,5,43,40,3,99
4,2,26,22,26,55,131
All,59,34,141,103,58,395


In [0]:
#Degrees of Freedom ----> DOF = (# rows_crosstab-1)*(# cols_crosstab-1)
total_rows=len(con.axes[0])
total_cols=len(con.axes[1])

dof = (total_rows - 1)*(total_cols-1)

print(dof)

16


In [0]:
#Expected Value Table
observed = con.values

observed

array([[ 2,  0,  1,  0,  0],
       [25,  1, 28,  5,  0],
       [22,  2, 47, 32,  0],
       [ 8,  5, 43, 40,  3],
       [ 2, 26, 22, 26, 55]])

In [0]:
col_totals = df['Mjob'].value_counts().sort_index().values

col_totals

array([ 59,  34, 141, 103,  58])

In [0]:
row_totals = df['Medu'].value_counts().sort_index().values

row_totals

array([  3,  59, 103,  99, 131])

In [0]:
sample_size = df.shape[0]

sample_size

395

In [0]:
expected = np.array([[0.0,0.0,0.0,0.0,0.0],
                     [0,0,0,0,0],
                     [0,0,0,0,0],
                     [0,0,0,0,0],
                     [0,0,0,0,0]])

for i, col in enumerate(col_totals):
  for j, row in enumerate(row_totals): 
    expected_value = (row*col / sample_size)
    # print(expected_value, i, j)
    expected[i][j] = expected_value

# Same error as Ryan. I'll transpose to flip the array into the right order

expected_new = np.array(expected).T

expected_new

array([[ 0.44810127,  0.25822785,  1.07088608,  0.78227848,  0.44050633],
       [ 8.81265823,  5.07848101, 21.06075949, 15.38481013,  8.66329114],
       [15.38481013,  8.86582278, 36.76708861, 26.85822785, 15.12405063],
       [14.78734177,  8.52151899, 35.33924051, 25.81518987, 14.53670886],
       [19.56708861, 11.27594937, 46.76202532, 34.15949367, 19.23544304]])

In [0]:
chi2 = ((observed - expected_new)**2 / expected_new).sum()

chi2

224.6843492089439

In [0]:
'''

I used the UIowa applet to calculate the p-value = 0

'''

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [0]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

**The sample mean x is a point estimate of the population mean μ. In other words, it gives some estimated facts about the population**

- What are hypothesis test? How are they useful?

**A test used to determine whether the hypothesis assumed for the sample of data stands true for the entire population or not. Simply, the hypothesis is an assumption which is tested to determine the relationship between two data sets.**


- What is a "null hypothesis?"

**The hypothesis that there is no significant difference between specified populations, any observed difference being due to sampling or experimental error.**

- What is a p-value and what does it represent?

** The p-value is the probability of obtaining results as extreme as the observed results of a statistical hypothesis test, assuming that the null hypothesis is correct. A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis.**

- What does it mean for something to be "statistically significant?"

**In principle, a statistically significant result (usually a difference) is a result that's not attributed to chance. More technically, it means that if the Null Hypothesis is true (which means there really is no difference), there's a low probability of getting a result that large or larger.**


- What is a test statistic and how does it relate to a p-value?



- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)